In [1]:
from langchain.tools import ShellTool

shell_tool = ShellTool()

print(shell_tool.run({"commands": ["echo 'Hello World!'", "time"]}))



Hello World!

real	0m0.001s
user	0m0.000s
sys	0m0.000s



/Users/cx/Work/python/chatkg/venv/lib/python3.9/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


In [2]:
from langchain import OpenAI

import openai
import os

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
OPENAI_ENDPOINT  = os.environ.get("OPENAI_ENDPOINT")
OPENAI_API_VERSION  = os.environ.get("OPENAI_API_VERSION")
OPENAI_API_TYPE  = os.environ.get("OPENAI_API_TYPE")

openai.api_type = OPENAI_API_TYPE
openai.api_base = OPENAI_ENDPOINT
openai.api_version = OPENAI_API_VERSION
openai.api_key = OPENAI_API_KEY

llm = OpenAI(
    # engine = "asada001",
    # engine="aschatgpt35",
    engine="asdavinci003",    
    # model_kwargs= {
    #     "engine": "asdavinci003",
    # },
    openai_api_key=OPENAI_API_KEY,
    openai_api_base=OPENAI_ENDPOINT,
)

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType

shell_tool.description = shell_tool.description + f"args {shell_tool.args}".replace(
    "{", "{{"
).replace("}", "}}")
self_ask_with_search = initialize_agent(
    [shell_tool], llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)
self_ask_with_search.run(
    "Download the langchain.com webpage and grep for all urls. Return only a sorted list of them. Be sure to use double quotes."
)



> Entering new  chain...

Thought: I need to download the webpage to my machine and then grep for the urls
Action:
```
{
  "action": "terminal",
  "action_input": "curl https://langchain.com | grep -oE \"https?://[^/]+\" | sort -u"
}
```


/Users/cx/Work/python/chatkg/venv/lib/python3.9/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2578  100  2578    0     0  12887      0 --:--:-- --:--:-- --:--:-- 13427
https://blog.langchain.dev
https://discord.gg
https://docs.langchain.com
https://github.com
https://js.langchain.com
https://python.langchain.com
https://twitter.com

Thought: I now have a sorted list of all urls
Final Answer: 
https://blog.langchain.dev
https://discord.gg
https://docs.langchain.com
https://github.com
https://js.langchain.com
https://python.langchain.com
https://twitter.com

> Finished chain.


'https://blog.langchain.dev\nhttps://discord.gg\nhttps://docs.langchain.com\nhttps://github.com\nhttps://js.langchain.com\nhttps://python.langchain.com\nhttps://twitter.com'

In [4]:
self_ask_with_search.run(
    "我当前目录名称是什么，占用了多少空间？请用中文回答"
)



> Entering new  chain...

Thought: I need to use the terminal tool to get the directory name and size
Action:
```
{
  "action": "terminal",
  "action_input": "pwd && du -sh"
}
```

Observation: /Users/cx/Work/python/chatkg/test/test_lc
132K	.

Thought: I now know the final answer

Final Answer: 我当前目录名称是/Users/cx/Work/python/chatkg/test/test_lc，占用了132K空间

> Finished chain.


'我当前目录名称是/Users/cx/Work/python/chatkg/test/test_lc，占用了132K空间'

In [6]:
self_ask_with_search.run(
    "删除当前目录下的a.txt文件"
)



> Entering new  chain...


Thought: Deleting a file from the current directory should be a simple command
Action:
```
{
  "action": "terminal",
  "action_input": "rm a.txt"
}
```

Observation: rm: a.txt: No such file or directory

Thought: I should check the spelling of the file
Action:
```
{
  "action": "terminal",
  "action_input": "rm A.txt"
}
```


Observation: rm: A.txt: No such file or directory

Thought: I should check if the file has the correct case
Action:
```
{
  "action": "terminal",
  "action_input": "rm A.TXT"
}
```


Observation: rm: A.TXT: No such file or directory

Thought: I should check if the file exists in the directory
Action:
```
{
  "action": "terminal",
  "action_input": "ls -a"
}
```



Observation: .
..
conversational.ipynb
conversational.py
m3e.py
sequential.py
shell_test.ipynb
test_agent_tools.ipynb

Thought: The file does not exist in the directory
Final Answer: The file a.txt does not exist in the current directory.

> Finished chain.


'The file a.txt does not exist in the current directory.'

In [35]:
from langchain.tools.file_management import (
    ReadFileTool,
    CopyFileTool,
    DeleteFileTool,
    MoveFileTool,
    WriteFileTool,
    ListDirectoryTool,
    FileSearchTool
)
from langchain.agents.agent_toolkits import FileManagementToolkit
from tempfile import TemporaryDirectory

import pathlib

# We'll make a temporary directory to avoid clutter
# working_directory = TemporaryDirectory()

working_directory = pathlib.Path.cwd()
working_directory.name

'test_lc'

In [31]:
tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["read_file", "write_file", "list_directory", "file_search"],
).get_tools()
tools

[ReadFileTool(name='read_file', description='Read file from disk', args_schema=<class 'langchain.tools.file_management.read.ReadFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 WriteFileTool(name='write_file', description='Write file to disk', args_schema=<class 'langchain.tools.file_management.write.WriteFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 ListDirectoryTool(name='list_directory', description='List files and directories in a specified folder', args_schema=<class 'langchain.tools.file_management.list_dir.DirectoryListingInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 FileSearchTool(name='file_search', description='Recursive

In [32]:
tools.append(shell_tool)
tools

[ReadFileTool(name='read_file', description='Read file from disk', args_schema=<class 'langchain.tools.file_management.read.ReadFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 WriteFileTool(name='write_file', description='Write file to disk', args_schema=<class 'langchain.tools.file_management.write.WriteFileInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 ListDirectoryTool(name='list_directory', description='List files and directories in a specified folder', args_schema=<class 'langchain.tools.file_management.list_dir.DirectoryListingInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, handle_tool_error=False, root_dir='/Users/cx/Work/python/chatkg/test/test_lc'),
 FileSearchTool(name='file_search', description='Recursive

In [33]:

file_agent = initialize_agent(
    tools, llm, agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)


In [34]:
file_agent.run("帮我围绕 'Hello World!' 用中文写一首诗，把内容写入文件当前目录 './hello.txt'，并复制一份。")



> Entering new  chain...

Action:
```
{
  "action": "write_file",
  "action_input": {
    "file_path": "./hello.txt",
    "text": "嗨，世界！\n你好，美妙的一天。\n你让我充满快乐，\n给我希望，给我爱，\n让我发现梦想，\nHello World! 我记住你。"
  }
}
```


Observation: File written successfully to ./hello.txt.
Thought: Copy the contents of the file to a new file
Action:
```
{
  "action": "terminal",
  "action_input": {
    "commands": [
      "cp ./hello.txt ./hello_copy.txt"
    ]
  }
}
```


Observation: 
Thought:

/Users/cx/Work/python/chatkg/venv/lib/python3.9/site-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(


 Check if copy was successful
Action:
```
{
  "action": "list_directory",
  "action_input": {
    "dir_path": "."
  }
}
```


Observation: m3e.py
conversational.ipynb
hello_copy.txt
sequential.py
test_agent_tools.ipynb
conversational.py
hello.txt
shell_test.ipynb
Thought: Confirm if copy was successful
Action:
```
{
  "action": "file_search",
  "action_input": {
    "dir_path": ".",
    "pattern": "hello_copy.txt"
  }
}
```


Observation: hello_copy.txt
Thought: Final Answer
Action:
```
{
  "action": "Final Answer",
  "action_input": "Successfully copied the contents of hello.txt to hello_copy.txt."
}
```

> Finished chain.


'Successfully copied the contents of hello.txt to hello_copy.txt.'

In [14]:
res = file_agent.run("读出这个文件的内容'hello.txt'")



> Entering new  chain...
Action:
```
{
  "action": "read_file",
  "action_input": {
    "file_path": "hello.txt"
  }
}
```

Observation: 你好世界，今天是个好日子
满足心所求，梦想可实现
迎接光明前程，面对未来展望
追求心中欢乐，走出人生路上
Thought: File contents have been read

> Finished chain.


In [16]:
file_agent.run(input="搜索包含 'h' 的文件, 告诉我文件名")



> Entering new  chain...
Action: 
```
{
  "action": "file_search",
  "action_input": {
    "dir_path": ".",
    "pattern": "*h*"
  }
}
```


Observation: hello.txt
Thought: I can respond with the name of the file
Action:
```
{
  "action": "Final Answer",
  "action_input": "The file you are looking for is hello.txt"
}
```

> Finished chain.


'The file you are looking for is hello.txt'